In [ ]:
!pip install --upgrade google-auth google-auth-oauthlib google-api-python-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 48.0 MB/s eta 0:00:00
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.27.0
    Uninstalling google-auth-2.27.0:
      Successfully uninstalled google-auth-2.27.0
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.155.0
    Uninstalling google-api-python-client-2.155.0:
      Successfully uninstalled google-api-python-client-2.155.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.27.0, but you have google-auth 2.37.0 which is incompatible.


WORKINGGGGGG CODEEEE


In [41]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os.path
import re

class MarkdownToGoogleDocs:


    SCOPES = ['https://www.googleapis.com/auth/documents']

    def __init__(self):
        self.credentials = None
        self.service = None

    def authenticate(self):
        """Handle Google Docs API authentication specifically for Colab"""
        try:
            auth.authenticate_user()
            self.credentials = GoogleCredentials.get_application_default()
            self.service = build('docs', 'v1', credentials=self.credentials)
            print("Authentication successful!")
        except Exception as e:
            raise Exception(f"Authentication failed: {str(e)}")

    def create_document(self, title):
        """Create a new Google Doc"""
        try:
            document = self.service.documents().create(body={'title': title}).execute()
            return document.get('documentId')
        except Exception as e:
            raise Exception(f"Error creating document: {str(e)}")


    def process_markdown(self, markdown_text):
      requests = []
      current_index = 1  # Start after title

      # Split content into lines
      lines = markdown_text.split('\n')

      for line in lines:
          # Skip empty lines
          if not line.strip():
              continue

          # Process headers
          if line.startswith('#'):
              level = len(re.match(r'^#+', line).group())
              text = line.lstrip('#').strip()

              requests.append({
                  'insertText': {
                      'location': {'index': current_index},
                      'text': text + '\n'
                  }
              })

              requests.append({
                  'updateParagraphStyle': {
                      'range': {
                          'startIndex': current_index,
                          'endIndex': current_index + len(text)
                      },
                      'paragraphStyle': {
                          'namedStyleType': f'HEADING_{level}',
                          'spaceAbove': {'magnitude': 10, 'unit': 'PT'},
                          'spaceBelow': {'magnitude': 10, 'unit': 'PT'}
                      },
                      'fields': 'namedStyleType,spaceAbove,spaceBelow'
                  }
              })

              current_index += len(text) + 1  # Correctly update current_index after header

          # Process checkboxes
          elif re.match(r'^- \[ \]', line):
              text = line[6:].strip()

              requests.append({
                  'insertText': {
                      'location': {'index': current_index},
                      'text': text + '\n'
                  }
              })

              requests.append({
                  'createParagraphBullets': {
                      'range': {
                          'startIndex': current_index,
                          'endIndex': current_index + len(text) + 1
                      },
                      'bulletPreset': 'BULLET_CHECKBOX'
                  }
              })

              # Style @mentions
              mentions = re.finditer(r'@(\w+)', text)
              for match in mentions:
                  requests.append({
                      'updateTextStyle': {
                          'range': {
                              'startIndex': current_index + match.start(),
                              'endIndex': current_index + match.end()
                          },
                          'textStyle': {
                              'bold': True,
                              'foregroundColor': {
                                  'color': {
                                      'rgbColor': {'red': 0.2, 'green': 0.4, 'blue': 0.8}
                                  }
                              }
                          },
                          'fields': 'bold,foregroundColor'
                      }
                  })

              current_index += len(text) + 1  # Update current_index after processing checkboxes

          # Process bullet points
          elif line.startswith('*') or line.startswith('-'):
              indent_level = (len(line) - len(line.lstrip())) // 2
              text = line.lstrip('* -').strip()

              requests.append({
                  'insertText': {
                      'location': {'index': current_index},
                      'text': text + '\n'
                  }
              })


              bullet_preset = 'BULLET_DISC_CIRCLE_SQUARE' if indent_level == 0 else 'BULLET_SQUARE'

              requests.append({
                  'createParagraphBullets': {
                      'range': {
                          'startIndex': current_index,
                          'endIndex': current_index + len(text) + 1
                      },
                      'bulletPreset': bullet_preset
                  }
              })

              current_index += len(text) + 1  # Update current_index after processing bullet points

          # Process regular text
          else:
              requests.append({
                  'insertText': {
                      'location': {'index': current_index},
                      'text': line + '\n'
                  }
              })
              current_index += len(line) + 1  # Update current_index after processing regular text

      return requests


    def update_document(self, document_id, requests):
        """Apply the formatting requests to the document"""
        try:
            # Execute requests in smaller batches to avoid potential issues
            batch_size = 20
            for i in range(0, len(requests), batch_size):
                batch = requests[i:i + batch_size]
                self.service.documents().batchUpdate(
                    documentId=document_id,
                    body={'requests': batch}
                ).execute()
        except Exception as e:
            raise Exception(f"Error updating document: {str(e)}")

#meeting notes provided in doc
markdown_text = """# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""

try:
    # Install required packages if not already installed
    !pip install --upgrade google-auth-oauthlib google-auth-httplib2 google-api-python-client

    # Create converter instance
    converter = MarkdownToGoogleDocs()

    # Authenticate (this will prompt for Google login)
    converter.authenticate()

    # Create new document and convert markdown
    doc_id = converter.create_document("Product Team Sync")
    requests = converter.process_markdown(markdown_text)
    converter.update_document(doc_id, requests)

    print(f"\nDocument created successfully!")
    print(f"View your document at: https://docs.google.com/document/d/{doc_id}")

except Exception as e:
    print(f"An error occurred: {str(e)}")

Authentication successful!

Document created successfully!
View your document at: https://docs.google.com/document/d/1sQVKg3CqEn9gEWRN18Ss7UetM8oe5cTkNbpdgcO9WQ4
